In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import math
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle, randint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [6]:
df = pd.read_csv('Breast_cancer_data.csv')
print(df.head())
print(df.shape)

   mean_radius  mean_texture  mean_perimeter  mean_area  mean_smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   diagnosis  
0          0  
1          0  
2          0  
3          0  
4          0  
(569, 6)


In [27]:
class GAHyperOpt:
    def __init__(self, features, target, data, params, model, generations, populationSize, mutationPercentage, elites):
        self.X = df[features]
        self.y = df[target]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.3)
        self.model=model
        self.params=params
        self.currPopulation=[]
        self.nextPopulation=[]
        self.generations=generations
        self.accArr=[]
        self.populationSize=populationSize
        self.mutationPercentage=mutationPercentage
    
    def createPopulation(self):
        for childNum in range(self.populationSize):
            myParams={}
            for i in self.params:
                decider=np.random.uniform(0,1)
                paramValue=i[1]+(decider*(i[2]-i[1]))
                if type(i[1])==float:
                    paramValue=round(paramValue, 2)
                elif type(i[1])==int:
                    paramValue=int(paramValue)
                myParams[i[0]]=paramValue
            self.currPopulation.append([myParams,self.fitness(myParams)])
        print("self.currPopulation",self.currPopulation)
        
    def fitness(self,params):
        self.model.set_params(**params)
        self.model.fit(self.X_train, self.y_train)
        self.y_pred=self.model.predict(self.X_test)
        return accuracy_score(self.y_pred, self.y_test)
            
    def mutate(self):
        for params in self.currPopulation:
            decider=randint(0,100)
            probability=100-self.mutationPercentage
            if decider >= probability:
                sign=np.random.choice([-1,1])
                pc=randint(0,len(params)-1)
                currValue=self.currParams[params[pc][0]]
                decider=np.random.uniform(0,n)
                paramValue=(sign*decider*(self.params[pc][2]-self.params[pc][1]))+currValue
                if type(self.params[pc][1])==float:
                    paramValue=min(max(round(paramValue, 2),self.params[pc][1]),self.params[pc][2])
                elif type(self.params[pc][1])==int:
                    paramValue=min(max(int(paramValue),self.params[pc][1]),self.params[pc][2])
                                        
#     def elitism(self):
#         for i in range(0,self.elites):

In [25]:
features=['mean_radius','mean_texture','mean_perimeter','mean_area','mean_smoothness']
target=['diagnosis']

params=[["num_leaves",2,500],
    ["max_depth",1,50],
    ["learning_rate",0.01,5],
    ["n_estimators",1,1000],
    ["colsample_bytree",0.01,1]]

myGenerations, myPopulationSize, myMutationPercentage, myElites = 10, 2, 2, 2

#         (self, features, target, data, params, model, generations, populationSize, mutationPercentage, elites)
myObj=GAHyperOpt(features, target, df, params, lgb.LGBMClassifier(), myGenerations, myPopulationSize, myMutationPercentage, myElites)

In [26]:
myObj.createPopulation()

self.currPopulation [[{'num_leaves': 89, 'max_depth': 25, 'learning_rate': 2.65, 'n_estimators': 189, 'colsample_bytree': 0.86}, 0.6491228070175439], [{'num_leaves': 301, 'max_depth': 30, 'learning_rate': 4.36, 'n_estimators': 292, 'colsample_bytree': 0.77}, 0.8538011695906432]]
